In [2]:
# import rocrate models
import os
import sys

#sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '../')))
#os.path.join(os.path.dirname(__file__), '../')
srcPath = os.path.abspath('C:\\Users\\Max\\Documents\\GitHub\\mds_python\\src\\' )
sys.path.insert(0, srcPath)

In [3]:
import pathlib
from fairscape_mds.config import create_fairscape_config
from dotenv import load_dotenv
import os

load_dotenv("../local.env")

fairscapeConfig = create_fairscape_config(os.environ)

#ldapConnection = fairscapeConfig.ldap.connectAdmin()
#currentUserLDAP = getUserByCN(ldapConnection, userCN)
#ldapConnection.unbind()

minioClient = fairscapeConfig.minio.CreateClient()
mongoClient = fairscapeConfig.mongo.CreateClient()

mongoDB = mongoClient[fairscapeConfig.mongo.db]
asyncCollection = mongoDB[fairscapeConfig.mongo.async_collection]
identifierCollection = mongoDB[fairscapeConfig.mongo.identifier_collection]
rocrateCollection = mongoDB[fairscapeConfig.mongo.rocrate_collection]

In [4]:
import fairscape_mds.models.rocrate as rocrate

c:\Users\Max\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [5]:
import timeit
import uuid
import pathlib
import shutil
import zipfile
import json
import minio
import urllib.parse

In [55]:
def uploadLocalFileMinio(
	minioClient: minio.Minio, 
	bucketName: str, 
	objectName: str, 
	filePath: str, 
	metadata: dict= None):
	uploadResult = minioClient.fput_object(
			bucket_name= bucketName,
			object_name=objectName,
			file_path=filePath,
			metadata=metadata
		)
	return uploadResult

## In Memory Approach

1. get transaction folder
2. extract metadata
3. validate metadata
4. extract all datasets
5. upload extracted datasets
6. mint metadata

### Seperate Job for updating archive

1. create transaction folder
2. unzip archive 
3. get metadata for rocrate

In [ ]:
# mock the transaction

# given rocrate
rocrateFolder = pathlib.Path("D:\Work\Data\Dataverse-Uploaded\crates_format_1.2")

testROCrate = rocrateFolder / "1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip"

# mock user CN
userCN = 'mal8ch'

# mock uuid
transactionID = str(uuid.uuid4())

# upload local crate to minio
uploadPath = pathlib.PurePosixPath(fairscapeConfig.minio.default_bucket_path) / userCN / 'rocrates' / str(transactionID) / testROCrate.name



In [9]:
mockUpload = minioClient.fput_object(
	bucket_name=fairscapeConfig.minio.default_bucket, 	
	object_name=str(uploadPath),
	file_path=str(testROCrate),
	metadata={"owner": userCN}
)

In [10]:
mockUpload.etag

'79cc44603e7c0395570276a51eb88f3d'

In [11]:
from zipfile import ZipFile
import json

In [14]:
# get transaction

# get object from minio 
objectRetrival = minioClient.get_object(
	bucket_name= fairscapeConfig.minio.default_bucket,
	object_name= str(uploadPath),
)


In [16]:
fairscapeConfig.minio

MinioConfig(host='localhost', port='9000', secret_key='miniotestsecret', access_key='miniotestadmin', default_bucket='default', default_bucket_path='default', rocrate_bucket='default', rocrate_bucket_path='rocrate', secure=False, cert_check=False)

In [13]:
import pydantic
import boto3
from botocore.client import Config

In [14]:
# Create S3 Client with support for reading files from Zips

s3 = boto3.client('s3',
	endpoint_url="http://localhost:9000",
	aws_access_key_id=fairscapeConfig.minio.access_key,
	aws_secret_access_key=fairscapeConfig.minio.secret_key,
	config=Config(signature_version='s3v4'),
	region_name='us-east-1'
)

def _add_header(request, **kwargs):
	request.headers.add_header('x-minio-extract', 'true')

event_system = s3.meta.events
event_system.register_first('before-sign.s3.*', _add_header)



In [15]:
# upload with headers x-minio-extract
s3.delete_object(
	Bucket=fairscapeConfig.minio.default_bucket,
	Key=str(uploadPath)	
)

with testROCrate.open('rb') as uploadFile:
	s3.upload_fileobj(
		Bucket=fairscapeConfig.minio.default_bucket,
		Key=str(uploadPath),
		Fileobj=uploadFile,
		ExtraArgs={'ContentType' :'application/zip'}
	)

In [16]:
# list zip contents
response = s3.list_objects_v2(
	Bucket=fairscapeConfig.minio.default_bucket,
	Prefix=str(uploadPath) + "/" 
	)

In [17]:
zipCrateContents = response['Contents']

In [19]:
zipCrateContents

[{'Key': 'default/mal8ch/uploads/fed6910d-6b1c-4052-8a74-a5fa6061edda/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/error.log',
  'LastModified': datetime.datetime(2025, 3, 11, 15, 49, 17, 871000, tzinfo=tzutc()),
  'ETag': '',
  'Size': 1237,
  'StorageClass': 'STANDARD'},
 {'Key': 'default/mal8ch/uploads/fed6910d-6b1c-4052-8a74-a5fa6061edda/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/output.log',
  'LastModified': datetime.datetime(2025, 3, 11, 15, 49, 17, 871000, tzinfo=tzutc()),
  'ETag': '',
  'Size': 10584,
  'StorageClass': 'STANDARD'},
 {'Key': 'default/mal8ch/uploads/fed6910d-6b1c-4052-8a74-a5fa6061edda/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_edgelist.tsv',
  'LastModified': datetim

In [18]:
zippedMetadataKey

NameError: name 'zippedMetadataKey' is not defined

In [167]:
# find the rocrate within a zip
zippedMetadataKey = uploadPath / uploadPath.name.rstrip('.zip') / 'ro-crate-metadata.json'

try:
	metadataResponse = s3.get_object(
		Bucket=fairscapeConfig.minio.default_bucket,
		Key=str(zippedMetadataKey),
	)

# TODO handle no such key error
except s3.exceptions.NoSuchKey as err:

	# Update Transaction
	print(err)


try:
	metadataContent = metadataResponse['Body'].read()
	metadataJSON = json.loads(metadataContent)

# TODO more specific exceptions
except:

	# TODO update the transaction
	pass

In [1]:
def cleanGUID(metadata):
	if "http" in metadata.guid:
		metadata.guid = urllib.parse.urlparse(metadata.guid).path.lstrip('/')

def cleanIdentifiers(crate: ROCrateV1_2):
	rocrateMetadata = crate.getCrateMetadata()

	#clean ROCrate metadata identifier
	cleanGUID(rocrateMetadata)

	
	# clean identifiers
	for elem in crate.getEVIElements():
		cleanGUID(elem)

		if isinstance(elem, ROCrateDataset):
			# usedByComputation
			for usedByComputation in elem.usedByComputation:
				cleanGUID(usedByComputation)
			
			# generatedBy
			for generatedBy in elem.generatedBy:
				cleanGUID(generatedBy)

		if isinstance(elem, ROCrateSoftware):
			for usedByElem in elem.usedByComputation:
				cleanGUID(usedByElem)

		if isinstance(elem, ROCrateComputation):
			#elem.usedDataset
			for usedDataset in elem.usedDataset:
				cleanGUID(usedDataset)
			#elem.generated
			for generated in elem.generated:
				cleanGUID(generated)
			#elem.usedSoftware
			for usedSoftware in elem.usedSoftware:
				cleanGUID(usedSoftware)


def processMetadataIsPartOf(crate: ROCrateV1_2):
	eviElements = crate.getEVIElements()
	rocrateRootElem = crateModel.getCrateMetadata()

	rocrateRootElem.hasPart = [ IdentifierValue.model_validate({"@id": x.guid}) for x in eviElements]

	# set inverse relation
	elemIsPartOf = IdentifierValue.model_validate({"@id": rocrateRootElem.guid})
	for elem in eviElements:
		elem.isPartOf = elemIsPartOf

def processMetadata(crate: ROCrateV1_2):
	pass

NameError: name 'ROCrateV1_2' is not defined

In [168]:
# validate json 
try:
	metadata = rocrate.ROCrateV1_2.model_validate(metadataJSON)

except pydantic.ValidationError as err:

	#TODO: handle pydantic validation errors


	#TODO: update transaction with metadata validation failure
	validationErrors = err.errors()
	pass

#


In [183]:
#zipCrateContents

In [ ]:

datasets = metadata.getDatasets()
testDataset = datasets[0]

In [190]:

# match dataset element to file inside zip
# TODO: deal with non unique matches

datasetFileKey = testDataset.contentUrl.lstrip('file:///')
datasetS3Match = next(filter(lambda x: datasetFileKey in x.get('Key'), zipCrateContents))

 
datasetS3Key = datasetS3Match['Key']
#[ datasetElem for datasetElem in zipCrateContents if datasetFileKey in datasetElem.get('Key')]

In [193]:
datasetS3Key

'default/mal8ch/uploads/78e44abb-f967-4067-bfed-5a43426f4e13/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_gene_node_attributes.tsv'

### Copy Object from Zip to other Minio Object

In [197]:
# Can't Copy Objects from within Zips
userDatasetPaths = pathlib.PurePosixPath(fairscapeConfig.minio.default_bucket_path) / userCN / 'datasets' 
newKey = userDatasetPaths / datasetFileKey

copy_source = {
	'Bucket': fairscapeConfig.minio.default_bucket,
	'Key': datasetS3Key
}

try:
	s3.copy_object(CopySource=copy_source, Bucket=fairscapeConfig.minio.default_bucket, Key=str(newKey))
except:
	pass



In [ ]:
# get file contents




In [ ]:
# upload new object

In [ ]:
# add distribution information to Dataset Element

## In Filesystem approach
1. create transaction folder
2. copy crate to local directory
3. unzip contents into transaction folder
4. process metadata
5. upload extracted datasets
6. replace ro-crate-metadata.json
7. zip up and upload archive

## Mocking the Transaction

1. create transaction folder
2. Copy crate to local directory
3. Unzip contents in transaction folder
4. process metadata
5. upload extracted datasets
6. replace ro-crate-metadata.json
7. zip up and upload archive

In [7]:
# mock user CN
userCN = 'mal8ch'

In [8]:
# mock a transaction uuid
transactionID = str(uuid.uuid4())

# create a folder for jobs
jobsFolder = pathlib.Path("jobs/")
jobsFolder.mkdir(exist_ok=True)

# create a folder for this transaction
transactionFolder = jobsFolder / transactionID
transactionFolder.mkdir(exist_ok=True)

In [9]:
extractFolder = transactionFolder / 'extracted'

In [10]:
# mock copy to transaction directory
zippedCratePath = transactionFolder / testROCrate.name

shutil.copyfile(
	str(testROCrate),
	str(zippedCratePath)
)


'jobs\\91e12cb3-f796-4203-ad40-f490f7b3006c\\1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha.zip'

In [11]:
crateFolderName = zippedCratePath.name.strip(zippedCratePath.suffix)

In [12]:
# unzip the archive 
with zippedCratePath.open('rb') as zippedCrateFileObj:
	zipCrate = zipfile.ZipFile(zippedCrateFileObj)
	zipCrate.extractall(path=str(extractFolder))

In [13]:
# extract the metadata
extractedContentsFolder = extractFolder / crateFolderName
metadataFilePath = extractedContentsFolder / 'ro-crate-metadata.json'
with metadataFilePath.open("r") as crateMetadataFileObj:
	crateMetadataJSON = json.load(crateMetadataFileObj)

# parse metadata into pydantic model
fullCrateModel = ROCrateV1_2.model_validate(crateMetadataJSON)

In [ ]:
# add schema models to ROCrateV1_2 pydantic model metadataGraph Union
# also look for schemas within the ROCrate before looking for Schemas in minio

In [14]:
def cleanGUID(metadata):
	if "http" in metadata.guid:
		metadata.guid = urllib.parse.urlparse(metadata.guid).path.lstrip('/')

def cleanIdentifiers(crate: ROCrateV1_2):
	rocrateMetadata = crate.getCrateMetadata()

	#clean ROCrate metadata identifier
	cleanGUID(rocrateMetadata)

	
	# clean identifiers
	for elem in crate.getEVIElements():
		cleanGUID(elem)

		if isinstance(elem, ROCrateDataset):
			# usedByComputation
			for usedByComputation in elem.usedByComputation:
				cleanGUID(usedByComputation)
			
			# generatedBy
			for generatedBy in elem.generatedBy:
				cleanGUID(generatedBy)

		if isinstance(elem, ROCrateSoftware):
			for usedByElem in elem.usedByComputation:
				cleanGUID(usedByElem)

		if isinstance(elem, ROCrateComputation):
			#elem.usedDataset
			for usedDataset in elem.usedDataset:
				cleanGUID(usedDataset)
			#elem.generated
			for generated in elem.generated:
				cleanGUID(generated)
			#elem.usedSoftware
			for usedSoftware in elem.usedSoftware:
				cleanGUID(usedSoftware)


def processMetadataIsPartOf(crate: ROCrateV1_2):
	eviElements = crate.getEVIElements()
	rocrateRootElem = crateModel.getCrateMetadata()

	rocrateRootElem.hasPart = [ IdentifierValue.model_validate({"@id": x.guid}) for x in eviElements]

	# set inverse relation
	elemIsPartOf = IdentifierValue.model_validate({"@id": rocrateRootElem.guid})
	for elem in eviElements:
		elem.isPartOf = elemIsPartOf

def processMetadata(crate: ROCrateV1_2):
	pass

In [19]:
fullCrateModel.model_dump_json(by_alias=True)

'{"@context":{"EVI":"https://w3id.org/EVI#","@vocab":"https://schema.org/"},"@graph":[{"@id":"ro-crate-metadata.json","@type":"CreativeWork","conformsTo":{"@id":"https://w3id.org/ro/crate/1.2-DRAFT"},"about":{"@id":"https://fairscape.net/ark:59852/rocrate-1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha"}},{"@id":"ark:59852/rocrate-1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha","@type":["Dataset","https://w3id.org/EVI#ROCrate"],"name":"Initial integration run","keywords":["Ideker Lab","CM4AI","0.1 alpha","MDA-MB-468","untreated","chromatin","Initial integration run","AP-MS edgelist download"],"isPartOf":[{"@id":"ark:/Ideker_Lab"},{"@id":"ark:/Ideker_Lab/CM4AI"}],"version":"0.5alpha","license":"https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en","associatedPublication":"Clark T, Schaffer L, Obernier K, Al Manir S, Churas CP, Dailamy A, Doctor Y, Forget A, Hansen JN, Hu M, Lenkiewicz J, Levinson MA, Marquez C, Mohan J, Nourreddine S, Nies

In [21]:
fullCrateModel.model_dump(by_alias=True)

{'@context': {'EVI': 'https://w3id.org/EVI#', '@vocab': 'https://schema.org/'},
 '@graph': [{'@id': 'ro-crate-metadata.json',
   '@type': 'CreativeWork',
   'conformsTo': {'@id': 'https://w3id.org/ro/crate/1.2-DRAFT'},
   'about': {'@id': 'https://fairscape.net/ark:59852/rocrate-1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha'}},
  {'@id': 'ark:59852/rocrate-1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha',
   '@type': ['Dataset', 'https://w3id.org/EVI#ROCrate'],
   'name': 'Initial integration run',
   'keywords': ['Ideker Lab',
    'CM4AI',
    '0.1 alpha',
    'MDA-MB-468',
    'untreated',
    'chromatin',
    'Initial integration run',
    'AP-MS edgelist download'],
   'isPartOf': [{'@id': 'ark:/Ideker_Lab'}, {'@id': 'ark:/Ideker_Lab/CM4AI'}],
   'version': '0.5alpha',
   'license': 'https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en',
   'associatedPublication': 'Clark T, Schaffer L, Obernier K, Al Manir S, Churas CP, Dailamy A, D

In [84]:
# for datasetElem in datasetList:

datasetElem = datasetList[0]
datasetContentUrl = datasetElem.contentUrl
if 'http' in datasetContentUrl:
	pass
elif 'file:' in datasetContentUrl:
	# create a path to the file specified by the dataset metadata 
	sourcePath = pathlib.Path(datasetElem.contentUrl.lstrip('file:///'))
	datasetFilePath = extractedContentsFolder / sourcePath.name

# check that path exists in the crate
if not datasetFilePath.exists():
	pass



uploadPath = pathlib.PurePosixPath(fairscapeConfig.minio.default_bucket_path) / userCN / 'datasets' / sourcePath

datasetUploadStart = time.time()
# upload the file 
uploadResult = uploadLocalFileMinio(
		minioClient=minioClient, 
		bucketName=fairscapeConfig.minio.default_bucket, 	
		objectName=str(uploadPath),
		filePath=str(datasetFilePath),
		metadata={"guid": datasetElem.guid, "owner": userCN}
		)

datasetUploadEnd = time.time()

uploadDuration = datasetUploadEnd - datasetUploadStart
print(uploadDuration)


# create a distribution for the dataset
datasetDistribution = FairscapeDataDistribution(
	distributionType = 'minio',
	objectPath=str(uploadPath),
	objectBucket=fairscapeConfig.minio.default_bucket
)


# set contentUrl in metadata to fairscape pointer
datasetElem.contentUrl = f"{fairscapeConfig.url}/dataset/download/{datasetElem.guid}"



0.007513523101806641


In [79]:
str(pathlib.PurePosixPath(uploadPath))

'default/mal8ch/datasets/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_gene_node_attributes.tsv'

WindowsPath('jobs/625d5589-4956-46a7-bc32-d6571dac285c/extracted/1.cm4ai_chromatin_mda-mb-468_untreated_apmsloader_initialrun0.1alpha/ppi_gene_node_attributes.tsv')

True

In [ ]:
# mint datasets